In [1]:
import numpy as np
from numpy import array, inf
from numpy.random import randint, randn, seed
from math import pi
from qiskit import(QuantumCircuit, execute, Aer, IBMQ, assemble, transpile)
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.providers.ibmq.job import job_monitor
from commpy.channelcoding.convcode import Trellis, conv_encode, viterbi_decode
import hashlib, hmac
import secrets
from Crypto.Cipher import Salsa20

print("Imports Successful")
# Uncomment if we need to use the actual quantum backends
# provider = IBMQ.load_account()
# provider.backends()

Imports Successful


In [67]:
# Helper functions for BB84 QKD - partially copied from https://qiskit.org/textbook/ch-algorithms/quantum-key-distribution.html
def encode_message(bits, bases):
    message = []
    for i in range(len(bases)):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

def measure_message(message, bases):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(len(bases)):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        qasm_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = qasm_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(len(a_bases)):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

# Original helper functions
def bits_to_byte_array(bits):
    if ((len(bits) % 8) != 0):
        raise RuntimeError('Bits length must be a multiple of 8')
    byte_str = "".join(str(bit) for bit in bits)
    return int(byte_str, 2).to_bytes(len(byte_str) // 8, byteorder='big')

def byte_array_to_bits(byte_arr):
    bits = []
    for i in byte_arr:
        binary = bin(i)[2:].zfill(8)
        for b in binary:
            bits.append(int(b))
    
    return bits

def bb84_qkd(desired_key_length, eavesdropper_detection_sample_size):
    if ((desired_key_length % 8) != 0):
        raise RuntimeError('Key length must be a multiple of 8')
        
    n = (desired_key_length * 2) + (eavesdropper_detection_sample_size * 6) 
    bob_key = []
    
    while (len(bob_key) < desired_key_length):
        alice_bits = randint(2, size=n)
        alice_bases = randint(2, size=n)

        # Pretend the message is transmitted here
        message = encode_message(alice_bits, alice_bases)

        bob_bases = randint(2, size=n)
        bob_results = measure_message(message, bob_bases)

        # Pretend Alice and Bob's bases were shared here
        alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
        bob_key = remove_garbage(alice_bases, bob_bases, bob_results)

        # Pretend the selection of random sample bits was shared here
        bit_selection = randint(n, size=eavesdropper_detection_sample_size)
        
        bob_sample = sample_bits(bob_key, bit_selection)
        alice_sample = sample_bits(alice_key, bit_selection)

        if (bob_sample != alice_sample):
            print("Eavesdropper detected! Reattempt key distribution on a different quantum channel.")
            raise RuntimeError
    return bits_to_byte_array(bob_key[0:desired_key_length])
  
def calculate_hash_bases(key, length):
    # Secure option: Produce <length> bits of data using the key and a fixed value with some cipher.
    # Paper option: x0+x4 (mod 2)
    # Simple option: Just use the key directly and loop through it until length is reached using key bits as 0 = +, 1 = X.
    hash_bases = []
    bit_key = byte_array_to_bits(key)
    for i in range(length):
        hash_bases.append(int(bit_key[i % len(bit_key)]))
    return hash_bases

def mix_message_and_hash(key, message, hashed):
    # Simplest option: Just concatenate the two, hash + message or alternate or something
    # Paper option: Uses only one hash bit and calculates sum(ki * 2^i mod n) for 0..n to determine its position
    # Better option: Hmmm - message is probably longer than the hash, but the hash length is fixed so len(message) = len(received) - len(hash). Use key to encrypt something of right length then ???
    mixed = []
    for x in hashed:
        mixed.append(x)
    for x in message:
        mixed.append(x)
    return mixed

def unmix_message_and_hash(key, mixed, hash_length):
    return (mixed[0:hash_length], mixed[hash_length:])

# Bob uses a special measurement function to measure in the omega basis
def bob_measure_message(message):
    backend = Aer.get_backend('qasm_simulator')
    measurements = []
    for q in range(len(message)):
        message[q].rz(-pi/8,0)
        message[q].measure(0,0)
        qasm_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = qasm_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

# Properties for convolutional ECC - Reference https://commpy.readthedocs.io/en/latest/channelcoding.html
x = 10
memory = array([x])
g_matrix = array([[5, 3, 7, 1]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

def convolutional_ecc_encode(message):
    msg_bits = byte_array_to_bits(message)
    coded_bits = conv_encode(msg_bits, ecc_trellis)
    return coded_bits
                        
def convolutional_ecc_decode(coded_bits):                    
    decoded_bits = viterbi_decode(coded_bits.astype(float), ecc_trellis)
    decoded_msg = bits_to_byte_array(decoded_bits[:-x])
    return decoded_msg
    
    
def ad_2018(message, preshared_key):
    # Salsa20 is a stream cipher which will basically take the short preshared key and expand it to a long one-time-pad equivalent for XORing
    cipher = Salsa20.new(key=preshared_key) 
    
    # This 8-byte nonce must be shared with Bob
    nonce = cipher.nonce
    encrypted = cipher.encrypt(message) 

    # Alice adds ECC to her message using a convolutional code
    alice_message = convolutional_ecc_encode(encrypted)

    # expanded_key = secrets.token_bytes(len(ecc_encoded))
    # alice_message = bytes(a ^ b for (a, b) in zip(ecc_encoded, preshared_key))

    # Alice prepares a keyed hash of her message
    hash_key = secrets.token_bytes(16) # Or bb84_qkd(128,16) or other protocol
    hash_function = hmac.new(hash_key, message, digestmod='md5') # Not a secure hash function but used here as it's a short hash
    alice_hash = hash_function.digest()

    # print(alice_message)
    # print("ECC Coded length: %i" % len(alice_message))
    # print(alice_hash)
    # print(len(alice_hash))

    # Alice mixes the bits of her message and hash according to some function f(k) based on the preshared key
    # then transmits message bits in random basis and hash bits in a basis given by g(k) based on the preshared key
    alice_message_bases = randint(2, size=len(alice_message))
    alice_hash_bases = calculate_hash_bases(preshared_key, len(alice_hash) * 8)

    alice_transmission_bases = mix_message_and_hash(preshared_key, alice_message_bases, alice_hash_bases)
    alice_transmission_bits = mix_message_and_hash(preshared_key, alice_message, byte_array_to_bits(alice_hash))
    # print(alice_transmission_bases)
    # print(alice_transmission_bits)

    encoded_message = encode_message(alice_transmission_bits, alice_transmission_bases)
    
    unmixed = unmix_message_and_hash(preshared_key, encoded_message, len(alice_hash)*8)
    hash_qubits = unmixed[0]
    message_qubits = unmixed[1]
    # print(hash_qubits)
    # print(message_qubits)
    bob_received_message = bob_measure_message(message_qubits)

    bob_hash_bases = calculate_hash_bases(preshared_key, len(alice_hash) * 8)
    bob_received_hash = bits_to_byte_array(measure_message(hash_qubits, bob_hash_bases))

    boblen = len(bob_received_message)
    for i in range(boblen-x, boblen):
        bob_received_message[i] = 0
    # print(array(bob_received_message))

    ecc_decoded = convolutional_ecc_decode(array(bob_received_message))
    # print(ecc_decoded)
    # print(encrypted)

    cipher = Salsa20.new(preshared_key, nonce=nonce)
    decrypted = cipher.decrypt(ecc_decoded)
    # print(decrypted)

    hash_function = hmac.new(hash_key, decrypted, digestmod='md5')
    bob_hash = hash_function.digest()
    # print(bob_hash)
    # print(bob_received_hash)
    match = hmac.compare_digest(bob_hash, bob_received_hash)
    # print("Is message verified? %s" % match)
    return match

In [3]:
# Simplified BB84 key exchange - call one method and it produces a byte key of the desired length
exchanged_key = bb84_qkd(128, 16)
print("Generated key is: %s" % exchanged_key)

Generated key is: b'j\x19m\xe93\x15\x02\x17\xef\xe0\x1aJ.\x9aE\xc6'


In [105]:
message = b'Hey'

# Using a preshared key produced with BB84 or some other key exchange protocol, Alice encrypts her message
preshared_key = bb84_qkd(128,16)
cipher = Salsa20.new(key=preshared_key) # Salsa20 is a stream cipher which will basically take the short preshared key and expand it to a long one-time-pad equivalent for XORing
# This 8-byte nonce must be shared with Bob
nonce = cipher.nonce
encrypted = cipher.encrypt(message) 
print(encrypted)

# Alice adds ECC to her message using a convolutional code
alice_message = convolutional_ecc_encode(encrypted)

# expanded_key = secrets.token_bytes(len(ecc_encoded))
# alice_message = bytes(a ^ b for (a, b) in zip(ecc_encoded, preshared_key))

# Alice prepares a keyed hash of her message
hash_key = secrets.token_bytes(16) # Or bb84_qkd(128,16) or other protocol
hash_function = hmac.new(hash_key, message, digestmod='md5') # Not a secure hash function but used here as it's a short hash
alice_hash = hash_function.digest()

# print(alice_message)
print("ECC Coded length: %i" % len(alice_message))
# print(alice_hash)
# print(len(alice_hash))

# Alice mixes the bits of her message and hash according to some function f(k) based on the preshared key
# then transmits message bits in random basis and hash bits in a basis given by g(k) based on the preshared key
alice_message_bases = randint(2, size=len(alice_message))
alice_hash_bases = calculate_hash_bases(preshared_key, len(alice_hash) * 8)

alice_transmission_bases = mix_message_and_hash(preshared_key, alice_message_bases, alice_hash_bases)
alice_transmission_bits = mix_message_and_hash(preshared_key, alice_message, byte_array_to_bits(alice_hash))
# print(alice_transmission_bases)
# print(alice_transmission_bits)

encoded_message = encode_message(alice_transmission_bits, alice_transmission_bases)
# Send encoded message to Bob!

b'S:\x14'
ECC Coded length: 324


In [43]:
unmixed = unmix_message_and_hash(preshared_key, encoded_message, len(alice_hash)*8)
hash_qubits = unmixed[0]
message_qubits = unmixed[1]
# print(hash_qubits)
# print(message_qubits)
bob_received_message = bob_measure_message(message_qubits)

bob_hash_bases = calculate_hash_bases(preshared_key, len(alice_hash) * 8)
bob_received_hash = bits_to_byte_array(measure_message(hash_qubits, bob_hash_bases))

boblen = len(bob_received_message)
for i in range(boblen-x, boblen):
    bob_received_message[i] = 0
# print(array(bob_received_message))

ecc_decoded = convolutional_ecc_decode(array(bob_received_message))
print(ecc_decoded)
print(encrypted)

cipher = Salsa20.new(preshared_key, nonce=nonce)
decrypted = cipher.decrypt(ecc_decoded)
print(decrypted)

hash_function = hmac.new(hash_key, decrypted, digestmod='md5')
bob_hash = hash_function.digest()
print(bob_hash)
print(bob_received_hash)
match = hmac.compare_digest(bob_hash, bob_received_hash)
print("Is message verified? %s" % match)


b'XfO'
b'8fO'
b'(ey'
b'd\r\xf2U\x81+\x05\xb2FEp\xaa\xbaa(k'
b'Z\x11\xf9\xe5\x9a\x87\x81\xfd\xc6?\xf1\xf2\xfe\x8b\xe7\x11'
Is message verified? False


In [ ]:
In the paper the authors use the generator matrix G = 
[[1,0,1,1,0],
 [0,1,1,0,1]]
This is a (5,2,3) code which can correct 1 error in a 5 length vector, so 20% woo. To calculate the encoding of 2 bits, we take
[0,1]*G = [x,x,x,x,x] using multiplication mod 2
So far so good, we can encode if we need to
Bob uses nearest neighbor decoding to go from his 5-bits back to 2

I am thinking that the paper is just incorrect about Bob's ability to decode 85% of qubits correctly as this is not observed in practice

In [100]:
def analyze_ad2018(count, message):
    preshared_key = bb84_qkd(128, 16)
    success = 0
    for i in range(count):
        if (ad_2018(message, preshared_key)):
            success += 1
            # print('success')
            
    print('%i/%i success rate' % (success, count))

In [78]:
x = 3
memory = array([x])
g_matrix = array([[7, 5, 3, 1]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hey')

247/1000 success rate


In [79]:
x = 3
memory = array([x])
g_matrix = array([[5, 7, 1, 3, 5]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hey')

334/1000 success rate


In [80]:
x = 5
memory = array([x])
g_matrix = array([[7, 5, 3, 1]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hey')

234/1000 success rate


In [81]:
x = 3
memory = array([x])
g_matrix = array([[7, 5, 3, 2]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hey')

251/1000 success rate


In [82]:
x = 3
memory = array([x])
g_matrix = array([[5, 7, 1, 3, 5, 1]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hey')

477/1000 success rate


In [84]:
x = 3
memory = array([x])
g_matrix = array([[5, 7, 1, 3, 5, 1, 3, 7, 6, 2]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hey')

815/1000 success rate


In [85]:
x = 5
memory = array([x])
g_matrix = array([[5, 7, 1, 3, 5, 1, 3, 7, 6, 2]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hey')

823/1000 success rate


In [86]:
x = 3
memory = array([x])
g_matrix = array([[5, 7, 1, 3, 5, 1, 3, 7, 3, 1]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hey')

844/1000 success rate


In [94]:
# Hello World
x = 3
memory = array([x])
g_matrix = array([[5, 7, 1, 3, 5, 1, 3, 7, 3, 1, 9, 5]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hello world')

582/1000 success rate


In [104]:
x = 3
memory = array([x])
g_matrix = array([[1, 3, 5, 7, 9, 11, 1, 3, 5, 7, 9, 11]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hey')

900/1000 success rate


In [96]:
# just Hello
x = 3
memory = array([x])
g_matrix = array([[5, 7, 1, 3, 5, 1, 3, 7, 3, 1, 9, 5]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hello')

799/1000 success rate


In [97]:
x = 3
memory = array([x])
g_matrix = array([[1, 3, 5, 7, 9, 11, 1, 3, 5, 7, 9, 11]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'hello')

843/1000 success rate


In [103]:
x = 3
memory = array([x])
g_matrix = array([[1, 3, 5, 7, 9, 11, 1, 3, 5, 7, 9, 11]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'1')
analyze_ad2018(1000, b'12')
analyze_ad2018(1000, b'123')
analyze_ad2018(1000, b'1234')
analyze_ad2018(1000, b'12345')
analyze_ad2018(1000, b'123456')
analyze_ad2018(1000, b'1234567')
analyze_ad2018(1000, b'12345678')

955/1000 success rate
926/1000 success rate
902/1000 success rate
853/1000 success rate
821/1000 success rate
800/1000 success rate
780/1000 success rate
756/1000 success rate


In [106]:
x = 3
memory = array([x])
g_matrix = array([[5, 7]])
ecc_trellis = Trellis(memory, g_matrix, code_type='default')

analyze_ad2018(1000, b'1')
analyze_ad2018(1000, b'12')
analyze_ad2018(1000, b'123')

262/1000 success rate
58/1000 success rate
9/1000 success rate
